# RNA Guide Editor Finder - Interactive Parser Tutorial

This notebook demonstrates all parser functions with interactive examples.
You can run each cell to see inputs and outputs.

## Setup

In [ ]:
# Import all necessary modules
import sys
import os
from pathlib import Path
import tempfile

# Add project to path
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

# Import parsers
from utils.parsers import (
    # Prodigal parsers
    parse_prodigal_faa,
    parse_prodigal_faa_full,
    # FASTA parsers
    parse_fasta,
    parse_fasta_records,
    # Diamond BLASTP parsers
    parse_diamond_blastp,
    parse_diamond_blastp_with_positions,
    # HMMER parsers
    parse_hmm_tblout,
    parse_hmm_tblout_with_positions,
    # Sequence retrieval
    retrieve_sequence_from_fasta,
    retrieve_sequences_from_hits,
    reverse_complement,
    # DataFrame conversion
    diamond_hits_to_dataframe,
    hmm_hits_to_dataframe,
    save_hits_to_csv,
)

import pandas as pd

print("✓ All modules imported successfully!")
print(f"Working directory: {project_root}")

## Create Sample Data Files

Let's create sample files to work with

In [ ]:
# Create a temporary directory for sample files
temp_dir = Path(tempfile.mkdtemp(prefix='parser_tutorial_'))
print(f"Created temporary directory: {temp_dir}")

# 1. Create sample FASTA file
fasta_file = temp_dir / "sample_genome.fna"
fasta_content = """>contig_1
ATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG
GCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTAGCTA
TACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACG
CGTAGCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA
>contig_2
AAAATTTTCCCCGGGGAAAATTTTCCCCGGGGAAAATTTTCCCCGGGGAAAATTTTCCCC
GGGGAAAATTTTCCCCGGGGAAAATTTTCCCCGGGGAAAATTTTCCCCGGGGAAAATTTT
"""
fasta_file.write_text(fasta_content)
print(f"✓ Created: {fasta_file.name}")

# 2. Create sample Prodigal FAA file
prodigal_faa = temp_dir / "proteins.faa"
prodigal_content = """>contig_1_1 # 100 # 300 # 1 # ID=1_1;partial=00;start_type=ATG;rbs_motif=None
MKLVPQRSTAVILGKLMNPQRSTAVILGKLMNPQ
>contig_1_2 # 500 # 800 # -1 # ID=1_2;partial=00;start_type=ATG;rbs_motif=GGA
MLKIPVRSTQAVILGKLMNPQRSTAVILGKLMN
>contig_2_1 # 50 # 250 # 1 # ID=2_1;partial=00;start_type=ATG;rbs_motif=None
MKLVPQRSTAVILGKLMNPQRSTAVILGKLMNPQRST
"""
prodigal_faa.write_text(prodigal_content)
print(f"✓ Created: {prodigal_faa.name}")

# 3. Create sample Diamond BLASTP results
diamond_file = temp_dir / "diamond_results.m8"
diamond_content = """contig_1_1\tIS110_transposase\t95.5\t200\t9\t0\t1\t200\t15\t214\t1.0e-100\t350.5
contig_1_2\tIS110_orfB\t88.2\t150\t18\t1\t1\t150\t10\t159\t2.3e-75\t280.3
contig_2_1\tDNA_polymerase\t92.0\t180\t14\t0\t1\t180\t5\t184\t5.1e-90\t320.1
"""
diamond_file.write_text(diamond_content)
print(f"✓ Created: {diamond_file.name}")

# 4. Create sample HMMER tblout results
hmmer_file = temp_dir / "hmmer_results.tbl"
hmmer_content = """# target name        accession   tlen query name           accession   qlen   E-value  score  bias   #  of  c-Evalue  i-Evalue  score  bias  from    to  from    to  from    to  acc description
contig_1_1           -            200 IS110_transposase    PF03400.15   250  1.5e-100  350.5   0.1   1   1  1.5e-100  1.5e-100  350.5   0.1     1   250     1   200     1   200 0.98 -
contig_1_2           -            150 IS110_orfB           PF12345.10   180  2.3e-75   280.3   0.2   1   1  2.3e-75   2.3e-75   280.3   0.2     1   180     1   150     1   150 0.95 -
contig_2_1           -            180 DNA_polymerase       PF00136.22   200  5.1e-90   320.1   0.0   1   1  5.1e-90   5.1e-90   320.1   0.0     1   200     1   180     1   180 0.99 -
"""
hmmer_file.write_text(hmmer_content)
print(f"✓ Created: {hmmer_file.name}")

print("\n✓ All sample files created!")

---
# Part 1: Prodigal Parsers

Parse Prodigal output to get protein positions

In [ ]:
# Show the input file
print("INPUT FILE (proteins.faa):")
print("=" * 70)
print(prodigal_faa.read_text())
print("=" * 70)

In [ ]:
# Parse positions only
print("FUNCTION: parse_prodigal_faa()")
print("=" * 70)

position_map = parse_prodigal_faa(str(prodigal_faa))

print("\nOUTPUT (position_map):")
print(f"Type: {type(position_map)}")
print(f"Number of proteins: {len(position_map)}")
print("\nContents:")
for protein_id, (start, end, strand) in position_map.items():
    strand_str = "forward" if strand == 1 else "reverse"
    print(f"  {protein_id}: {start}..{end} ({strand_str})")

In [ ]:
# Parse full information including sequences
print("FUNCTION: parse_prodigal_faa_full()")
print("=" * 70)

genes = parse_prodigal_faa_full(str(prodigal_faa))

print("\nOUTPUT (genes):")
print(f"Type: {type(genes)}")
print(f"Number of genes: {len(genes)}")

print("\nFirst gene details:")
first_gene = genes['contig_1_1']
print(f"  protein_id: {first_gene.protein_id}")
print(f"  start: {first_gene.start}")
print(f"  end: {first_gene.end}")
print(f"  strand: {first_gene.strand}")
print(f"  length: {first_gene.length}")
print(f"  is_forward: {first_gene.is_forward}")
print(f"  sequence: {first_gene.sequence[:30]}...")

---
# Part 2: FASTA Parsers

Parse FASTA files to get genomic sequences

In [ ]:
# Show the input file
print("INPUT FILE (sample_genome.fna):")
print("=" * 70)
print(fasta_file.read_text())
print("=" * 70)

In [ ]:
# Parse as dictionary
print("FUNCTION: parse_fasta()")
print("=" * 70)

sequences = parse_fasta(str(fasta_file))

print("\nOUTPUT (sequences):")
print(f"Type: {type(sequences)}")
print(f"Number of sequences: {len(sequences)}")

for seq_id, sequence in sequences.items():
    print(f"\n  {seq_id}:")
    print(f"    Length: {len(sequence)} bp")
    print(f"    First 60 bp: {sequence[:60]}")
    print(f"    Last 60 bp: {sequence[-60:]}")

In [ ]:
# Parse as records
print("FUNCTION: parse_fasta_records()")
print("=" * 70)

records = parse_fasta_records(str(fasta_file))

print("\nOUTPUT (records):")
print(f"Type: {type(records)}")
print(f"Number of records: {len(records)}")

for record in records:
    print(f"\n  Record:")
    print(f"    ID: {record.id}")
    print(f"    Header: {record.header}")
    print(f"    Length: {record.length} bp")
    print(f"    Sequence: {record.sequence[:60]}...")

---
# Part 3: Diamond BLASTP Parsers

Parse Diamond BLASTP results with genomic positions

In [ ]:
# Show the input files
print("INPUT FILE 1 (diamond_results.m8):")
print("=" * 70)
print(diamond_file.read_text())
print("=" * 70)

print("\nINPUT 2 (position_map from Prodigal):")
print("=" * 70)
for protein_id, (start, end, strand) in position_map.items():
    print(f"  {protein_id}: ({start}, {end}, {strand})")
print("=" * 70)

In [ ]:
# Parse basic Diamond results (without positions)
print("FUNCTION: parse_diamond_blastp()")
print("=" * 70)

basic_hits = parse_diamond_blastp(str(diamond_file))

print("\nOUTPUT (basic_hits):")
print(f"Type: {type(basic_hits)}")
print(f"Number of hits: {len(basic_hits)}")

print("\nFirst hit (tuple):")
print(f"  {basic_hits[0]}")

print("\nParsed fields:")
hit = basic_hits[0]
print(f"  query_id: {hit[0]}")
print(f"  subject_id: {hit[1]}")
print(f"  pident: {hit[2]}%")
print(f"  alignment_length: {hit[3]}")
print(f"  evalue: {hit[10]}")
print(f"  bitscore: {hit[11]}")

In [ ]:
# Parse Diamond results WITH positions
print("FUNCTION: parse_diamond_blastp_with_positions()")
print("=" * 70)

hits = parse_diamond_blastp_with_positions(
    str(diamond_file),
    position_map,
    file_basename="sample_genome"
)

print("\nOUTPUT (hits):")
print(f"Type: {type(hits)}")
print(f"Number of hits: {len(hits)}")

print("\nFirst hit (DiamondBlastHit object):")
hit = hits[0]
print(f"  file_basename: {hit.file_basename}")
print(f"  contig_id: {hit.contig_id}")
print(f"  protein_id: {hit.protein_id}")
print(f"  start: {hit.start}")
print(f"  end: {hit.end}")
print(f"  strand: {hit.strand}")
print(f"  is_forward: {hit.is_forward}")
print(f"  length: {hit.length} bp")
print(f"  query_id: {hit.query_id}")
print(f"  subject_id: {hit.subject_id}")
print(f"  pident: {hit.pident}%")
print(f"  evalue: {hit.evalue}")
print(f"  bitscore: {hit.bitscore}")

---
# Part 4: HMMER Parsers

Parse HMMER tblout results with genomic positions

In [ ]:
# Show the input file
print("INPUT FILE (hmmer_results.tbl):")
print("=" * 70)
print(hmmer_file.read_text())
print("=" * 70)

In [ ]:
# Parse basic HMMER results
print("FUNCTION: parse_hmm_tblout()")
print("=" * 70)

basic_hmm_hits = parse_hmm_tblout(str(hmmer_file))

print("\nOUTPUT (basic_hmm_hits):")
print(f"Type: {type(basic_hmm_hits)}")
print(f"Number of hits: {len(basic_hmm_hits)}")

print("\nFirst hit (tuple):")
hit = basic_hmm_hits[0]
print(f"  target_name: {hit[0]}")
print(f"  query_name: {hit[1]}")
print(f"  accession: {hit[2]}")
print(f"  evalue: {hit[3]}")
print(f"  score: {hit[4]}")
print(f"  bias: {hit[5]}")

In [ ]:
# Parse HMMER results WITH positions
print("FUNCTION: parse_hmm_tblout_with_positions()")
print("=" * 70)

hmm_hits = parse_hmm_tblout_with_positions(
    str(hmmer_file),
    position_map,
    file_basename="sample_genome"
)

print("\nOUTPUT (hmm_hits):")
print(f"Type: {type(hmm_hits)}")
print(f"Number of hits: {len(hmm_hits)}")

print("\nFirst hit (HmmHit object):")
hit = hmm_hits[0]
print(f"  file_basename: {hit.file_basename}")
print(f"  contig_id: {hit.contig_id}")
print(f"  protein_id: {hit.protein_id}")
print(f"  start: {hit.start}")
print(f"  end: {hit.end}")
print(f"  strand: {hit.strand}")
print(f"  is_forward: {hit.is_forward}")
print(f"  length: {hit.length} bp")
print(f"  query_name: {hit.query_name}")
print(f"  target_name: {hit.target_name}")
print(f"  evalue: {hit.evalue}")
print(f"  score: {hit.score}")
print(f"  bias: {hit.bias}")

---
# Part 5: Sequence Retrieval

Retrieve actual genomic sequences from FASTA files

In [ ]:
# Test reverse complement
print("FUNCTION: reverse_complement()")
print("=" * 70)

test_seq = "ATCGATCG"
rev_comp = reverse_complement(test_seq)

print(f"\nINPUT:  {test_seq}")
print(f"OUTPUT: {rev_comp}")

test_seq2 = "AAAATTTTCCCCGGGG"
rev_comp2 = reverse_complement(test_seq2)

print(f"\nINPUT:  {test_seq2}")
print(f"OUTPUT: {rev_comp2}")

In [ ]:
# Retrieve a specific sequence
print("FUNCTION: retrieve_sequence_from_fasta()")
print("=" * 70)

print("\nINPUT:")
print(f"  FASTA file: {fasta_file.name}")
print(f"  contig_id: contig_1")
print(f"  start: 1")
print(f"  end: 30")
print(f"  strand: 1 (forward)")

seq_forward = retrieve_sequence_from_fasta(
    str(fasta_file),
    contig_id="contig_1",
    start=1,
    end=30,
    strand=1
)

print(f"\nOUTPUT (forward):")
print(f"  {seq_forward}")

print("\n" + "=" * 70)
print("\nINPUT (reverse strand):")
print(f"  strand: -1 (reverse)")

seq_reverse = retrieve_sequence_from_fasta(
    str(fasta_file),
    contig_id="contig_1",
    start=1,
    end=30,
    strand=-1
)

print(f"\nOUTPUT (reverse - automatically reverse complemented):")
print(f"  {seq_reverse}")

In [ ]:
# Retrieve sequences for all Diamond hits
print("FUNCTION: retrieve_sequences_from_hits()")
print("=" * 70)

print("\nINPUT:")
print(f"  Number of hits: {len(hits)}")
print(f"  FASTA files: [{fasta_file.name}]")

# Before retrieval
print("\nBEFORE retrieval:")
for i, hit in enumerate(hits):
    has_seq = hasattr(hit, 'genomic_sequence') and hit.genomic_sequence
    print(f"  Hit {i+1}: has_sequence = {has_seq}")

# Retrieve sequences
hits_with_seq = retrieve_sequences_from_hits(
    hits,
    [str(fasta_file)],
    verbose=True
)

print("\nAFTER retrieval:")
for i, hit in enumerate(hits_with_seq):
    has_seq = hasattr(hit, 'genomic_sequence') and hit.genomic_sequence
    if has_seq:
        seq_len = len(hit.genomic_sequence)
        seq_preview = hit.genomic_sequence[:40] if len(hit.genomic_sequence) > 40 else hit.genomic_sequence
        print(f"  Hit {i+1}:")
        print(f"    protein_id: {hit.protein_id}")
        print(f"    position: {hit.contig_id}:{hit.start}-{hit.end}")
        print(f"    strand: {'forward' if hit.is_forward else 'reverse'}")
        print(f"    sequence_length: {seq_len} bp")
        print(f"    sequence: {seq_preview}...")
    else:
        print(f"  Hit {i+1}: NO SEQUENCE (contig not found)")

---
# Part 6: DataFrame Conversion

Convert hits to pandas DataFrames for analysis

In [ ]:
# Convert Diamond hits to DataFrame
print("FUNCTION: diamond_hits_to_dataframe()")
print("=" * 70)

print("\nINPUT:")
print(f"  Number of hits: {len(hits_with_seq)}")
print(f"  Type: List[DiamondBlastHit]")

df_diamond = diamond_hits_to_dataframe(hits_with_seq)

print("\nOUTPUT:")
print(f"  Type: {type(df_diamond)}")
print(f"  Shape: {df_diamond.shape}")
print(f"  Columns: {df_diamond.columns.tolist()}")

print("\n" + "=" * 70)
print("DataFrame content:")
print("=" * 70)
df_diamond

In [ ]:
# Show specific columns
print("Selected columns from DataFrame:")
print("=" * 70)
df_diamond[['protein_id', 'subject_id', 'pident', 'evalue', 'bitscore', 'is_forward']]

In [ ]:
# DataFrame info
print("DataFrame info:")
print("=" * 70)
df_diamond.info()

In [ ]:
# Convert HMMER hits to DataFrame
print("FUNCTION: hmm_hits_to_dataframe()")
print("=" * 70)

df_hmmer = hmm_hits_to_dataframe(hmm_hits)

print("\nOUTPUT:")
print(f"  Type: {type(df_hmmer)}")
print(f"  Shape: {df_hmmer.shape}")

print("\n" + "=" * 70)
print("DataFrame content:")
print("=" * 70)
df_hmmer

---
# Part 7: DataFrame Analysis Examples

Practical examples of analyzing the data

In [ ]:
# Filter by E-value
print("FILTER: E-value < 1e-80")
print("=" * 70)

high_confidence = df_diamond[df_diamond['evalue'] < 1e-80]

print(f"\nOriginal hits: {len(df_diamond)}")
print(f"Filtered hits: {len(high_confidence)}")
print("\nFiltered results:")
high_confidence[['protein_id', 'subject_id', 'evalue', 'pident']]

In [ ]:
# Filter by identity
print("FILTER: Identity > 90%")
print("=" * 70)

high_identity = df_diamond[df_diamond['pident'] > 90]

print(f"\nHits with >90% identity: {len(high_identity)}")
high_identity[['protein_id', 'subject_id', 'pident', 'evalue']]

In [ ]:
# Complex filtering (multiple conditions)
print("FILTER: E-value < 1e-80 AND Identity > 90% AND Forward strand")
print("=" * 70)

filtered = df_diamond[
    (df_diamond['evalue'] < 1e-80) &
    (df_diamond['pident'] > 90) &
    (df_diamond['is_forward'])
]

print(f"\nFiltered hits: {len(filtered)}")
filtered[['protein_id', 'subject_id', 'pident', 'evalue', 'is_forward']]

In [ ]:
# Sort by E-value
print("SORT: By E-value (ascending)")
print("=" * 70)

sorted_df = df_diamond.sort_values('evalue')
sorted_df[['protein_id', 'subject_id', 'evalue', 'bitscore']]

In [ ]:
# Group by subject
print("GROUP BY: subject_id")
print("=" * 70)

by_subject = df_diamond.groupby('subject_id').agg({
    'protein_id': 'count',
    'pident': 'mean',
    'evalue': 'min',
    'bitscore': 'max'
})
by_subject.columns = ['count', 'mean_pident', 'min_evalue', 'max_bitscore']

print("\nResults:")
by_subject

In [ ]:
# Statistics
print("STATISTICS: Summary statistics")
print("=" * 70)

print("\nNumeric columns:")
df_diamond[['pident', 'evalue', 'bitscore', 'length']].describe()

In [ ]:
# Count by category
print("VALUE COUNTS")
print("=" * 70)

print("\nHits per subject:")
print(df_diamond['subject_id'].value_counts())

print("\nHits per strand:")
print(df_diamond['is_forward'].value_counts())

---
# Part 8: Export Results

Save results to files

In [ ]:
# Export to CSV
print("EXPORT: Save to CSV")
print("=" * 70)

csv_file = temp_dir / "diamond_results.csv"

df_diamond.to_csv(csv_file, index=False)

print(f"\n✓ Saved to: {csv_file}")
print(f"  File size: {csv_file.stat().st_size} bytes")
print(f"  Rows: {len(df_diamond)}")
print(f"  Columns: {len(df_diamond.columns)}")

In [ ]:
# Read it back
print("READ: Load CSV file")
print("=" * 70)

df_loaded = pd.read_csv(csv_file)

print(f"\nLoaded DataFrame:")
print(f"  Shape: {df_loaded.shape}")
print("\nFirst few rows:")
df_loaded.head()

In [ ]:
# Use built-in save function
print("FUNCTION: save_hits_to_csv()")
print("=" * 70)

csv_file2 = temp_dir / "diamond_results_v2.csv"

save_hits_to_csv(hits_with_seq, str(csv_file2), include_sequence=True)

print(f"\n✓ File created: {csv_file2.name}")

---
# Part 9: Complete Workflow Example

Putting it all together

In [ ]:
print("COMPLETE WORKFLOW")
print("=" * 70)

# Step 1: Parse Prodigal
print("\nStep 1: Parse Prodigal FAA")
position_map = parse_prodigal_faa(str(prodigal_faa))
print(f"  ✓ Found {len(position_map)} proteins")

# Step 2: Parse Diamond BLASTP
print("\nStep 2: Parse Diamond BLASTP with positions")
hits = parse_diamond_blastp_with_positions(
    str(diamond_file),
    position_map,
    file_basename="sample_genome"
)
print(f"  ✓ Found {len(hits)} hits")

# Step 3: Retrieve sequences
print("\nStep 3: Retrieve genomic sequences")
hits = retrieve_sequences_from_hits(hits, [str(fasta_file)], verbose=False)
with_seq = sum(1 for h in hits if hasattr(h, 'genomic_sequence') and h.genomic_sequence)
print(f"  ✓ Retrieved sequences for {with_seq}/{len(hits)} hits")

# Step 4: Convert to DataFrame
print("\nStep 4: Convert to DataFrame")
df = diamond_hits_to_dataframe(hits)
print(f"  ✓ Created DataFrame with shape {df.shape}")

# Step 5: Filter
print("\nStep 5: Filter high-confidence hits")
high_conf = df[(df['evalue'] < 1e-80) & (df['pident'] > 90)]
print(f"  ✓ {len(high_conf)} hits pass filter")

# Step 6: Analyze
print("\nStep 6: Group by subject")
summary = df.groupby('subject_id')['pident'].mean()
print("  ✓ Summary statistics:")
for subject, mean_id in summary.items():
    print(f"    {subject}: {mean_id:.1f}% mean identity")

# Step 7: Export
print("\nStep 7: Export results")
output_file = temp_dir / "final_results.csv"
df.to_csv(output_file, index=False)
print(f"  ✓ Saved to {output_file.name}")

print("\n" + "=" * 70)
print("✓ WORKFLOW COMPLETE!")
print("=" * 70)

---
# Summary

You've now seen all the major functions in action:

1. **Prodigal Parsers** - Get protein positions
2. **FASTA Parsers** - Get genomic sequences
3. **Diamond BLASTP Parsers** - Parse BLAST results with positions
4. **HMMER Parsers** - Parse HMM results with positions
5. **Sequence Retrieval** - Get actual DNA sequences
6. **DataFrame Conversion** - Convert to pandas for analysis
7. **Analysis** - Filter, sort, group, aggregate
8. **Export** - Save results to CSV/Excel

## Next Steps

- Modify the examples with your own data
- Try different filters and analyses
- Combine multiple genomes
- Create visualizations with matplotlib/seaborn

## Cleanup

In [ ]:
# Clean up temporary files
import shutil

print(f"Temporary directory: {temp_dir}")
print(f"Files created: {len(list(temp_dir.glob('*')))}")

# Uncomment to delete
# shutil.rmtree(temp_dir)
# print("✓ Temporary files cleaned up")